In [44]:
import os, glob
import pydra
from pydra import Workflow
from pydra.engine.specs import File, MultiInputFile
import typing as ty
from pathlib import Path

# get current directory
pydra_tutorial_dir = os.path.dirname(os.getcwd())

# set up output directory
workflow_dir = Path(pydra_tutorial_dir) / 'outputs'
workflow_out_dir = workflow_dir / '7_glm'
output_dir = workflow_out_dir / 'test_nilearn'

# create the output directory if not exit
os.makedirs(output_dir, exist_ok=True)

## Read data

In [2]:
fmriprep_path = workflow_out_dir / 'data'
rawdata_path = workflow_out_dir / 'raw_data'
# get events.tsv list
event_list = glob.glob(os.path.join(rawdata_path, '*', 'func', '*events.tsv'))
event_list.sort()
# get img list
img_list = glob.glob(os.path.join(fmriprep_path, '*', 'func', '*space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz'))
img_list.sort()
# get mask list
mask_list  = glob.glob(os.path.join(fmriprep_path, '*', 'func', '*space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz'))
mask_list.sort()
# get confounds list
confound_list = glob.glob(os.path.join(fmriprep_path, '*', 'func', '*desc-confounds_timeseries.tsv'))
confound_list.sort()

In [108]:
def get_subj_file(subj_id, n_run, event_list, img_list, mask_list):
    # subj_id starts from 0
    start = subj_id*n_run
    end = start+2
    subj_events = event_list[start:end]
    subj_imgs = img_list[start:end]
    subj_masks = mask_list[start:end]
    return subj_id, subj_events, subj_imgs, subj_masks

In [109]:
def get_firstlevel_dm(tr, n_scans, hrf_model, subj_id, subj_imgs, subj_events):
    import numpy as np
    import pandas as pd
    from nilearn.glm.first_level import make_first_level_design_matrix
    from nilearn.interfaces.fmriprep import load_confounds_strategy
    # read event file
    events = []
    imgs = []
    for run_event in subj_events:
        event = pd.read_csv(run_event, sep='\t').fillna(0)
        event = event[['onset', 'duration', 'trial_type']]
        events.append(event)
    
    # get list of confounds directly from fmriprepped bold
    confounds = load_confounds_strategy(subj_imgs, denoise_strategy='simple')[0]
    
    frame_times = np.arange(n_scans) * tr
    design_matrices = []
    dm_paths = []
    for index, (ev, conf) in enumerate(zip(events, confounds)):
        design_matrix = make_first_level_design_matrix(frame_times, ev, 
                                                       hrf_model=hrf_model,
                                                       add_regs=conf)          
        
        # make sure all design matrices have the same length of column
        # if you have a block design, this is not needed.
        # 39 = 4(events) + 34(confounds) + 13(drift) + 1(constant)
        assert design_matrix.shape[1] == 52, "This design matrix has the wrong column number"
        # sort the column order alphabetical for contrasts
        design_matrix = design_matrix.reindex(sorted(design_matrix.columns), axis=1)
        dm_path = os.path.join(output_dir, 'sub-%s_run-%s_designmatrix.csv' % (subj_id, index+1))
        design_matrix.to_csv(dm_path, index=None)
        design_matrices.append(design_matrix)
        dm_paths.append(dm_path)
    return design_matrices, dm_paths

In [110]:
def set_contrast(subj_id, design_matrices):
    import pandas as pd
    import numpy as np
    from nilearn.plotting import plot_contrast_matrix
    
    design_matrix = design_matrices[0]
    contrast_matrix = np.eye(design_matrix.shape[1])
    basic_contrasts = dict([(column, contrast_matrix[i])
                      for i, column in enumerate(design_matrix.columns)])
    contrasts = {
        'pumps-control': basic_contrasts['pumps_demean'] - basic_contrasts['control_pumps_demean'],
        'control-pumps': -basic_contrasts['control_pumps_demean'] + basic_contrasts['pumps_demean'],
        'pumps-baseline': basic_contrasts['pumps_demean'],
        'cash-baseline': basic_contrasts['cash_demean'],
        'explode-baseline': basic_contrasts['explode_demean'],
        'effects_of_interest': np.vstack((basic_contrasts['pumps_demean'],
                                          basic_contrasts['cash_demean'],
                                          basic_contrasts['explode_demean']))
        }
    
    contrast_plot = []
    for index, (contrast_id, contrast_val) in enumerate(contrasts.items()):
        print('  Plot Contrast % 2i out of %i: %s' % (
            index + 1, len(contrasts), contrast_id))
        contrast_plot_path = os.path.join(output_dir, 'sub-%s_firstlevel_contrast-%s.jpg' % (subj_id, contrast_id))
        plot_contrast_matrix(contrast_val, design_matrix, output_file=contrast_plot_path)
        contrast_plot.append(contrast_plot_path)
    return contrasts, contrast_plot

In [111]:
def firstlevel_estimation(subj_id, subj_imgs, subj_masks, smoothing_fwhm, design_matrices, contrasts):
    import nibabel as nib
    from nilearn.image import concat_imgs, mean_img
    from nilearn.glm.first_level import FirstLevelModel
    
    print('Compute firstlevel mask...')
    # average mask across three runs
    mean_mask = math_img('np.mean(img, axis=-1)', img=subj_masks)
    # binarize the mean mask
    mask = math_img('img > 0', img=mean_mask)
    # fit the (fixed-effects) firstlevel model with three runs simultaneously
    first_level_model = FirstLevelModel(mask_img=mask, smoothing_fwhm=smoothing_fwhm, minimize_memory=True)
    first_level_model = first_level_model.fit(subj_imgs, design_matrices=design_matrices)
    
    print('Computing contrasts...')
    z_map_path_dict = dict.fromkeys(contrasts.keys())
    for index, (contrast_id, contrast_val) in enumerate(contrasts.items()):
        print('  Contrast % 2i out of %i: %s' % (
            index + 1, len(contrasts), contrast_id))
        # Estimate the contasts. Note that the model implicitly computes a fixed
        # effect across the two sessions
        z_map = first_level_model.compute_contrast(
            contrast_val, output_type='z_score')

        # write the resulting stat images to file
        z_map_path = os.path.join(output_dir, 'sub-%s_contrast-%s_z_map.nii.gz' % (subj_id, contrast_id))
        z_map_path_dict[contrast_id] = z_map_path
        z_map.to_filename(z_map_path)
        
    return first_level_model, z_map_path_dict

In [112]:
# select one subject
subj_list = [1,2,3]
n_run = 3
tr = 2.3
n_scans = 300
hrf_model='glover'
smoothing_fwhm=5.0
second_level_input = []
z_map_dict_list = []
for subj_id in subj_list:
    print('firstlevel GLM for sub-%s' % subj_id)
    subj_id, subj_events, subj_imgs, subj_masks = get_subj_file(
        subj_id=subj_id, n_run=n_run, 
        event_list=event_list, img_list=img_list, mask_list=mask_list)
    design_matrices, dm_paths = get_firstlevel_dm(
        tr=tr, n_scans=n_scans, hrf_model=hrf_model, 
    subj_id=subj_id, subj_imgs=subj_imgs, subj_events=subj_events)
    contrasts, contrast_plot = set_contrast(subj_id=subj_id, design_matrices=design_matrices)
    first_level_model, z_map_path_dict = firstlevel_estimation(
        subj_id=subj_id, subj_imgs=subj_imgs, subj_masks=subj_masks, 
        smoothing_fwhm=smoothing_fwhm, design_matrices=design_matrices, contrasts=contrasts)
    second_level_input.append(first_level_model)
    z_map_dict_list.append(z_map_path_dict)

firstlevel GLM for sub-1
  Plot Contrast  1 out of 6: pumps-control
  Plot Contrast  2 out of 6: control-pumps
  Plot Contrast  3 out of 6: pumps-baseline
  Plot Contrast  4 out of 6: cash-baseline
  Plot Contrast  5 out of 6: explode-baseline
  Plot Contrast  6 out of 6: effects_of_interest
Compute firstlevel mask...
Computing contrasts...
  Contrast  1 out of 6: pumps-control


/Users/yibeichen/miniconda3/envs/pydra/lib/python3.7/site-packages/nilearn/glm/first_level/first_level.py:657: UserWarning: One contrast given, assuming it for all 2 runs
  warn('One contrast given, assuming it for all %d runs' % n_runs)


  Contrast  2 out of 6: control-pumps
  Contrast  3 out of 6: pumps-baseline
  Contrast  4 out of 6: cash-baseline
  Contrast  5 out of 6: explode-baseline
  Contrast  6 out of 6: effects_of_interest


/Users/yibeichen/miniconda3/envs/pydra/lib/python3.7/site-packages/nilearn/glm/contrasts.py:352: UserWarning: Running approximate fixed effects on F statistics.
  warn('Running approximate fixed effects on F statistics.')


firstlevel GLM for sub-2
  Plot Contrast  1 out of 6: pumps-control
  Plot Contrast  2 out of 6: control-pumps
  Plot Contrast  3 out of 6: pumps-baseline
  Plot Contrast  4 out of 6: cash-baseline
  Plot Contrast  5 out of 6: explode-baseline
  Plot Contrast  6 out of 6: effects_of_interest
Compute firstlevel mask...
Computing contrasts...
  Contrast  1 out of 6: pumps-control


...

/Users/yibeichen/miniconda3/envs/pydra/lib/python3.7/site-packages/nilearn/glm/contrasts.py:352: UserWarning: Running approximate fixed effects on F statistics.
  warn('Running approximate fixed effects on F statistics.')


firstlevel GLM for sub-3
  Plot Contrast  1 out of 6: pumps-control
  Plot Contrast  2 out of 6: control-pumps
  Plot Contrast  3 out of 6: pumps-baseline
  Plot Contrast  4 out of 6: cash-baseline
  Plot Contrast  5 out of 6: explode-baseline
  Plot Contrast  6 out of 6: effects_of_interest
Compute firstlevel mask...
Computing contrasts...
  Contrast  1 out of 6: pumps-control


/Users/yibeichen/miniconda3/envs/pydra/lib/python3.7/site-packages/nilearn/glm/first_level/first_level.py:657: UserWarning: One contrast given, assuming it for all 2 runs
  warn('One contrast given, assuming it for all %d runs' % n_runs)


  Contrast  2 out of 6: control-pumps
  Contrast  3 out of 6: pumps-baseline
  Contrast  4 out of 6: cash-baseline
  Contrast  5 out of 6: explode-baseline
  Contrast  6 out of 6: effects_of_interest


/Users/yibeichen/miniconda3/envs/pydra/lib/python3.7/site-packages/nilearn/glm/contrasts.py:352: UserWarning: Running approximate fixed effects on F statistics.
  warn('Running approximate fixed effects on F statistics.')


## Second level GLM

In [100]:
def get_secondlevel_dm(subj_list):
    import pandas as pd
    n_subj = len(subj_list)
    design_matrix = pd.DataFrame([1] * n_subj,columns=['intercept'])
    dm_path = os.path.join(output_dir, 'secondlevel_designmatrix.csv')
    design_matrix.to_csv(dm_path, index=None)
    return design_matrix

In [124]:
def secondlevel_estimation(second_level_input, design_matrix, first_level_contrast):
    """ task to estimate the second level
    Parameters
    ----------
    second_level_input : list
        the list of FirstLevelModel
    design_matrix : ty.Any
        a pandas.DataFrame that specifies the second level design
    first_level_contrast : dict
        a dictionary of contrasts

    Returns
    -------
    second_level_model : SecondLevelModel object
        
    stat_maps_dict : dict
        
    """
    from nilearn.glm.second_level import SecondLevelModel
    second_level_model = SecondLevelModel().fit(second_level_input=second_level_input, confounds = None, design_matrix=design_matrix)
    
    print('Computing contrasts...')
    stat_maps_dict = {}
    for index, (contrast_id, contrast_val) in enumerate(first_level_contrast.items()):
        print('  Contrast % 2i out of %i: %s' % (
            index + 1, len(first_level_contrast), contrast_id))
        
        # Estimate the contasts. Note that the model implicitly computes a fixed
        # effect across the two sessions
        stat_maps = second_level_model.compute_contrast(first_level_contrast=contrast_val, output_type='all')
        stat_maps_dict[contrast_id] = stat_maps
        # # write the resulting stat images to file
        # z_image_path = path.join(output_dir, 'contrast-%s_z_map.nii.gz' % contrast_id)
        # z_image_path_list.append(z_image_path)
        # z_map.to_filename(z_image_path)
    return second_level_model, stat_maps_dict

In [151]:
def cluster_thresholding(stat_maps_dict, threshold, cluster_threshold):
    from nilearn.image import threshold_img
    from nilearn import plotting
    thresholded_map_dict = dict.fromkeys(stat_maps_dict.keys())
    plot_contrast_dict = dict.fromkeys(stat_maps_dict.keys())
    for index, (stats_id, stats_val) in enumerate(stat_maps_dict.items()):
        print('  Contrast % 2i out of %i: %s' % (
            index + 1, len(stat_maps_dict), stats_id))
        thresholded_map = threshold_img(
            img = stats_val['z_score'],
            threshold=threshold,
            cluster_threshold=cluster_threshold,
            two_sided=True,
        )
        thresholded_map_path = os.path.join(output_dir, 'secondlevel_cluster_thresholded_contrast-%s_z_map.nii.gz' % stats_id)
        thresholded_map_dict[stats_id] = thresholded_map_path
        thresholded_map.to_filename(thresholded_map_path)
        plot_path = os.path.join(output_dir, 
                                   'secondlevel_cluster_thresholded_contrast-%s_zmap.jpg' % stats_id)
        plot_contrast_dict[stats_id] = plot_path
        plotting.plot_stat_map(thresholded_map,
                               title='Cluster Thresholded z map',
                               output_file=plot_path)
    return thresholded_map_dict, plot_contrast_dict

In [155]:
def multiple_comparison(stat_maps_dict, alpha, height_control):
    from nilearn.glm import threshold_stats_img
    from nilearn import plotting
    thresholded_map_dict = dict.fromkeys(stat_maps_dict.keys())
    plot_contrast_dict = dict.fromkeys(stat_maps_dict.keys())
    for index, (stats_id, stats_val) in enumerate(stat_maps_dict.items()):
        print('  Contrast % 2i out of %i: %s' % (
            index + 1, len(stat_maps_dict), stats_id))
        thresholded_map, threshold = threshold_stats_img(
            stat_img=stats_val['z_score'], 
            alpha=alpha, 
            height_control=height_control)
        thresholded_map_path = os.path.join(output_dir, 
                                         'secondlevel_multiple_comp_corrected_contrast-%s_z_map.nii.gz' % stats_id)
        thresholded_map_dict[stats_id] = thresholded_map_path
        thresholded_map.to_filename(thresholded_map_path)
        plot_path = os.path.join(output_dir, 
                                   'secondlevel_multiple_comp_corrected_contrast-%s_zmap.jpg' % stats_id)
        plot_contrast_dict[stats_id] = plot_path
        plotting.plot_stat_map(thresholded_map,
                               title='Thresholded z map, expected fdr = .05',
                               threshold=threshold, 
                               output_file=plot_path)
    return thresholded_map_dict, plot_contrast_dict

In [163]:
def parametric_test(stat_maps_dict, second_level_model):
    import numpy as np
    from nilearn.image import get_data, math_img
    from nilearn import plotting
    thresholded_map_dict = dict.fromkeys(stat_maps_dict.keys())
    plot_contrast_dict = dict.fromkeys(stat_maps_dict.keys())
    for index, (stats_id, stats_val) in enumerate(stat_maps_dict.items()):
        print('  Contrast % 2i out of %i: %s' % (
            index + 1, len(stat_maps_dict), stats_id))
        p_val = stats_val['p_value']
        n_voxels = np.sum(get_data(second_level_model.masker_.mask_img_))
        # Correcting the p-values for multiple testing and taking negative logarithm
        neg_log_pval = math_img("-np.log10(np.minimum(1, img * {}))"
                                .format(str(n_voxels)),
                                img=p_val)
        
        thresholded_map_path = os.path.join(output_dir, 'secondlevel_paramatric_thresholded_contrast-%s_z_map.nii.gz' % stats_id)
        thresholded_map_dict[stats_id] = thresholded_map_path
        neg_log_pval.to_filename(thresholded_map_path)
    
        # Since we are plotting negative log p-values and using a threshold equal to 1,
        # it corresponds to corrected p-values lower than 10%, meaning that there is
        # less than 10% probability to make a single false discovery (90% chance that
        # we make no false discovery at all).  This threshold is much more conservative
        # than the previous one.
        title = ('parametric test (FWER < 10%)')
        plot_path = os.path.join(output_dir, 
                                   'secondlevel_paramatric_thresholded_contrast-%s_zmap.jpg' % stats_id)
        plot_contrast_dict[stats_id] = plot_path
        plotting.plot_glass_brain(
            neg_log_pval, colorbar=True, display_mode='z', plot_abs=False, 
            vmax=3, threshold=1, title=title, output_file=plot_path)
    return thresholded_map_dict, plot_contrast_dict

In [176]:
def nonparametric_test(z_map_dict_list, smoothing_fwhm, design_matrix, firstlevel_contrast, n_perm):
    """ task to estimate the second level
    Parameters
    ----------
    z_map_dict_list : list
        the list of first-level output (dictionary)
    design_matrix : ty.Any
        a pandas.DataFrame that specifies the second level design
    first_level_contrast : dict
        a dictionary of contrasts used in the first level
    n_perm: int
        number of permutation

    Returns
    -------
    thresholded_map_dict : dict
        
    plot_contrast_dict : dict
        
    """
    from nilearn.glm.second_level import non_parametric_inference
    from nilearn import plotting
    z_map_dict = {con: [sub[con] for sub in z_map_dict_list] for con in firstlevel_contrast.keys() }
    thresholded_map_dict = dict.fromkeys(z_map_dict.keys())
    plot_contrast_dict = dict.fromkeys(z_map_dict.keys())
    for index, (stats_id, stats_val) in enumerate(z_map_dict.items()):
        print('  Contrast % 2i out of %i: %s' % (
            index + 1, len(z_map_dict), stats_id))
        
        # here we set threshold as none to do voxel-level FWER-correction.
        neg_log_pvals_permuted_ols_unmasked = \
            non_parametric_inference(stats_val, design_matrix=design_matrix,
                                     model_intercept=True, n_perm=n_perm,
                                     two_sided_test=False, smoothing_fwhm=smoothing_fwhm, n_jobs=-2)
        
        thresholded_map_path = os.path.join(output_dir, 'secondlevel_permutation_contrast-%s_z_map.nii.gz' % stats_id)
        thresholded_map_dict[stats_id] = thresholded_map_path
        neg_log_pvals_permuted_ols_unmasked.to_filename(thresholded_map_path)
        # here I actually have more than one contrast
        title = ('permutation test (FWER < 10%)')
        plot_path = os.path.join(output_dir, 'secondlevel_permutation_contrast-%s_zmap.jpg' % stats_id)
        plot_contrast_dict[stats_id] = plot_path
        display = plotting.plot_glass_brain(
            neg_log_pvals_permuted_ols_unmasked, colorbar=True, vmax=3,
            display_mode='z', plot_abs=False, threshold=1, 
            title=title, output_file=plot_path)
    return thresholded_map_dict, plot_contrast_dict

In [156]:
design_matrix = get_secondlevel_dm(subj_list=subj_list)

In [138]:
second_level_model, stat_maps_dict = secondlevel_estimation(
    second_level_input=second_level_input, 
    design_matrix=design_matrix, 
    first_level_contrast=contrasts)

Computing contrasts...
  Contrast  1 out of 6: pumps-control
  Contrast  2 out of 6: control-pumps
  Contrast  3 out of 6: pumps-baseline
  Contrast  4 out of 6: cash-baseline
  Contrast  5 out of 6: explode-baseline
  Contrast  6 out of 6: effects_of_interest


In [153]:
threshold = 2.3
cluster_threshold = 10
thresholded_map_dict, plot_contrast_dict = cluster_thresholding(
    stat_maps_dict=stat_maps_dict, threshold=threshold, cluster_threshold=cluster_threshold)

  Contrast  1 out of 6: pumps-control
  Contrast  2 out of 6: control-pumps
  Contrast  3 out of 6: pumps-baseline
  Contrast  4 out of 6: cash-baseline
  Contrast  5 out of 6: explode-baseline
  Contrast  6 out of 6: effects_of_interest


In [157]:
alpha = 0.05
height_control = 'fdr'
thresholded_map_dict, plot_contrast_dict = multiple_comparison(
    stat_maps_dict=stat_maps_dict, alpha=alpha, height_control=height_control)

  Contrast  1 out of 6: pumps-control
  Contrast  2 out of 6: control-pumps
  Contrast  3 out of 6: pumps-baseline
  Contrast  4 out of 6: cash-baseline
  Contrast  5 out of 6: explode-baseline
  Contrast  6 out of 6: effects_of_interest


In [164]:
thresholded_map_dict, plot_contrast_dict = parametric_test(
    stat_maps_dict=stat_maps_dict, second_level_model=second_level_model)

  Contrast  1 out of 6: pumps-control


<string>:1: RuntimeWarning: divide by zero encountered in log10
/Users/yibeichen/miniconda3/envs/pydra/lib/python3.7/site-packages/nilearn/_utils/niimg.py:62: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  "Non-finite values detected. "


  Contrast  2 out of 6: control-pumps


<string>:1: RuntimeWarning: divide by zero encountered in log10


  Contrast  3 out of 6: pumps-baseline


...

<string>:1: RuntimeWarning: divide by zero encountered in log10


  Contrast  5 out of 6: explode-baseline


<string>:1: RuntimeWarning: divide by zero encountered in log10


  Contrast  6 out of 6: effects_of_interest


<string>:1: RuntimeWarning: divide by zero encountered in log10


In [177]:
thresholded_map_dict, plot_contrast_dict = nonparametric_test(
    z_map_dict_list=z_map_dict_list, smoothing_fwhm=smoothing_fwhm, 
    design_matrix=design_matrix, firstlevel_contrast=contrasts, n_perm=1)

  Contrast  1 out of 6: pumps-control
  Contrast  2 out of 6: control-pumps
  Contrast  3 out of 6: pumps-baseline
  Contrast  4 out of 6: cash-baseline
  Contrast  5 out of 6: explode-baseline
  Contrast  6 out of 6: effects_of_interest
